# whoosh and pandas

building a search database from a dataframe.

In [1]:
df = pandas.read_json("../../../research/at.json.gz")

df = df.assign(id=(h := df.index.to_series().apply(
        compose_left(str.encode, hashlib.sha256, operator.methodcaller("hexdigest"), take(10), "".join)
))).reset_index().set_index("id")

df.update(df.tags.dropna().apply(", ".join))
df = df.rename(columns=dict(tags="tag"))

In [2]:
import whoosh.index, whoosh.qparser

Path("a11yhood-search").mkdir(exist_ok=True)
index = whoosh.index.create_in("a11yhood-search", (
    whoosh.fields.Schema(
        id=whoosh.fields.ID(stored=True), name=whoosh.fields.TEXT, description=whoosh.fields.TEXT, 
        tag=whoosh.fields.KEYWORD
    )
))
if "writer" in locals(): 
    try: writer.cancel()
    except: ...
writer = index.writer()


df.reset_index()["id name description tag".split()].apply(
    do(lambda x: writer.add_document(**x.dropna().to_dict())), axis=1
)

writer.commit()

In [3]:
%%
there is a basic query parser while the `whoosh.qparser.MultifieldParser` provides significantly more flexible query composition.
it searches across multiple fields allowing for direct access to tags, ids, and description alike.

    q = "name:xac_mario_kart"
    q = "id:6797431caf id:9e3563d58e"
    q = "6797431caf 9e3563d58e"
    q = "wheel chair"
    q = "tag:'assistive technology'"
    with index.searcher() as search:
        parser = whoosh.qparser.MultifieldParser("name description tags id".split(), index.schema, group=whoosh.qparser.OrGroup)
        results = search.search(parser.parse(q))
        results = DataFrame(map(dict, results))
        if len(results):
            results = df.loc[results.id]

{% filter pug %}
p there are !{len(results)} results from for the search term
    samp !{q}
{% endfilter %}

{{results.style._repr_html_()}}

        display(results)

,index,description,stargazerCount,forkCount,license,tag,name
id,,,,,,,
122f1ec85c,https://www.thingiverse.com/thing:6841790,People with tetraplegia or other forms of limi...,NaN,NaN,Creative Commons - Attribution - Non-Commercial,"accessibility, assistive, AssistiveTech, assis...",Finger Blade for People with Hand Impairments
7b99152738,https://www.thingiverse.com/thing:3635665,"The Raindrop is small, simple DIY assistive sw...",NaN,NaN,Creative Commons - Attribution - Share Alike,"accessibility, AssistiveTech, assistive device...",Raindrop Assistive Switch
454e78389e,https://www.thingiverse.com/thing:6652475,[Disclaimer]\r\n\r\n It's my opinion this ...,NaN,NaN,Creative Commons - Attribution - Non-Commercia...,"AssistiveTech, assistive device, assistive tec...",Plug Puller - Velcro Strap Version [Use With C...
90bcb1f40a,https://www.thingiverse.com/thing:6652483,[Safety Warning!]\r\n\r\n {Under no circums...,NaN,NaN,Creative Commons - Attribution - Non-Commercia...,"AssistiveTech, assistive device, assistive tec...",Plug Puller V4 - Heavy Duty - [PETG Only!]
0c5c7a5550,https://www.thingiverse.com/thing:2870410,As I was working on a master’s degree in speci...,NaN,NaN,Creative Commons - Attribution,"AssistiveTech, assistive device, assistive tec...",Volksswitch - the People's AT Switch (Version 1)
2b27ecbfc4,https://www.thingiverse.com/thing:6652479,[Safety Warning!]\r\n\r\n {Under no circums...,NaN,NaN,Creative Commons - Attribution - Non-Commercia...,"AssistiveTech, assistive device, assistive tec...",Plug Puller V4 - [PETG Only!]
6cdf031fb2,https://www.thingiverse.com/thing:1097330,As an entry for the Assistive Technology chall...,NaN,NaN,Creative Commons - Attribution,"AssistiveTech, assistive device, assistive tec...",Saliva Collecting Mouthpiece
51e5270881,https://www.thingiverse.com/thing:3085183,This is the Project Vive button used for assis...,NaN,NaN,Creative Commons - Attribution,"AssistiveTech, assistive device, assistive tec...",Project Vive 3D printed Switch
e46f7b09af,https://www.thingiverse.com/thing:1871831,This is my design for a 3-D printable assistiv...,NaN,NaN,Creative Commons - Attribution - Share Alike,"3dprintable, 3dprinting, accessibility, Assist...",Open Assistive Technology - Bag Holder


there is a basic query parser while the whoosh.qparser.MultifieldParser provides significantly more flexible query composition.
it searches across multiple fields allowing for direct access to tags, ids, and description alike. 
 q = "name:xac_mario_kart"
q = "id:6797431caf id:9e3563d58e"
q = "6797431caf 9e3563d58e"
q = "wheel chair"
q = "tag:'assistive technology'"
with index.searcher() as search:
 parser = whoosh.qparser.MultifieldParser("name description tags id".split(), index.schema, group=whoosh.qparser.OrGroup)
 results = search.search(parser.parse(q))
 results = DataFrame(map(dict, results))
 if len(results):
 results = df.loc[results.id]
 
 there are 10 results from for the search term tag:'assistive technology' 
 

 
 
 
   
 index 
 description 
 stargazerCount 
 forkCount 
 license 
 tag 
 name 
 
 
 id 
   
   
   
   
   
   
   
 
 
 
 
 122f1ec85c 
 https://www.thingiverse.com/thing:6841790 
 People with tetraplegia or other forms of limited hand mobility often struggle to use traditional utility knives to open packages or cut materials, as these tools require strength and fine motor skills that they may not possess.
 The developed assistive solution is an innovative thimble with an integrated blade which can be 3D-printed. The thimble is worn over the finger, and the blade is securely fixed at the tip. The blade is positioned in such a way that it can be used safely and with control. Users do not need fine motor skills to operate the adapted cutter, as minimal finger movement activates the blade. Additionally, there is an integrated retraction mechanism to prevent injuries when the thimble is not in use. 
 We invite you to download and use this design to make everyday tasks a little easier for people with disabilities. 
 You can learn more about us and our projects by visiting our website: https://www.uibk.ac.at/de/projects/innklusion/ 
 #INNklusion 
 nan 
 nan 
 Creative Commons - Attribution - Non-Commercial 
 accessibility, assistive, AssistiveTech, assistive device, assistive device for elderly or disabled, assistive tech, assistive technology, disabilities, disability, Disability aid, disabled, hand tools, inclusion, inclusive, kitchen tool, technology, tool, tools, utility, utility blade, utility knife 
 Finger Blade for People with Hand Impairments 
 
 
 7b99152738 
 https://www.thingiverse.com/thing:3635665 
 The Raindrop is small, simple DIY assistive switch. The activation surface is 20 mm in diameter and the activation force is about 1.3N (130 gf). It has a similar size and activation force as the AbleNet Mini Cup Switch, at a fraction of the cost. 
 More details can be found at the MMC Open-Source Assistive Technology library project page:
 https://www.makersmakingchange.com/project/raindrop-switch/ 
 
 
 
 
 
 Volunteer your 3D printing skills to help people with disabilities in your community! 
 Makers Making Change is an initiative of the Neil Squire Society, a Canadian non-profit that helps people with disabilities. 
 We are committed to creating a network of volunteer makers who support people with disabilities in their communities through 3D printing assistive devices. Check out our library of free, open-source assistive technologies with parts and build instructions. 
 
 
 nan 
 nan 
 Creative Commons - Attribution - Share Alike 
 accessibility, AssistiveTech, assistive device, assistive tech, assistive technology, at switch, switch, xbox adaptive 
 Raindrop Assistive Switch 
 
 
 454e78389e 
 https://www.thingiverse.com/thing:6652475 
 [Disclaimer]
 It's my opinion this Plug Puller - Velcro Strap version is inferior to my previous model that uses zip ties. This was created and uploaded due to repeated suggestion from community members who wanted an alternative to the zip tie version. Despite my explanation, there was still a large amount of requests for a Velcro strap edit. So, I have uploaded this model in hopes they can discover why this is not a good solution.
 
 When designing assistive 